# DFTpy for bulk Al

## March  28, 2023

### Import needed DFTpy modules

In [1]:
import numpy as np

from dftpy.grid import DirectGrid
from dftpy.field import DirectField
from dftpy.functional import Functional, TotalFunctional
from dftpy.functional.abstract_functional import AbstractFunctional
from dftpy.functional.functional_output import FunctionalOutput
from dftpy.optimization import Optimization
from dftpy.constants import LEN_CONV, ENERGY_CONV
from dftpy.ions import Ions
from dftpy.math_utils import ecut2nr

In [2]:
from ase.build import bulk
from ase.io import read

In [3]:
from ase.spacegroup import crystal

In [4]:
import os
os.chdir('../..')
path_file = os.getcwd()

In [5]:
import h5py
file = h5py.File(path_file+"/organize_data/Data_phases.hdf5", "r")
fileX = file["Descriptor"]
rho0_KE_WT = file["rho0_KE_wt"]
rho0_TE_WT = file["rho0_TE_wt"]
rho0_DEN_WT = file["rho0_DEN_wt"]

rho0_KE_wt =[]
rho0_TE_wt =[]
rho0_DEN_wt=[]
rho = []
for i in np.arange(0,280):
    rho0_KE_wt.append(rho0_KE_WT[i][0]) 
    rho0_TE_wt.append(rho0_TE_WT[i][0]) 
    rho0_DEN_wt.append(rho0_DEN_WT[i][0])
file.close()
rho0_Btin_KE_wt = rho0_KE_wt[0:30];rho0_fcc_KE_wt = rho0_KE_wt[30:60];rho0_bcc_KE_wt = rho0_KE_wt[60:90];rho0_cd_KE_wt = rho0_KE_wt[90:120];rho0_dhcp_KE_wt = rho0_KE_wt[120:150];rho0_Btin_hd_KE_wt = rho0_KE_wt[150:180];rho0_rand4_KE_wt = rho0_KE_wt[180:185];rho0_rand8_KE_wt = rho0_KE_wt[185:190];rho0_bct5_KE_wt = rho0_KE_wt[190:220];rho0_hcp_KE_wt = rho0_KE_wt[220:250];rho0_sh_KE_wt = rho0_KE_wt[250:280]

rho0_Btin_TE_wt = rho0_TE_wt[0:30];rho0_fcc_TE_wt = rho0_TE_wt[30:60];rho0_bcc_TE_wt = rho0_TE_wt[60:90];rho0_cd_TE_wt = rho0_TE_wt[90:120];rho0_dhcp_TE_wt = rho0_TE_wt[120:150];rho0_Btin_hd_TE_wt = rho0_TE_wt[150:180];rho0_rand4_TE_wt = rho0_TE_wt[180:185];rho0_rand8_TE_wt = rho0_TE_wt[185:190];rho0_bct5_TE_wt = rho0_TE_wt[190:220];rho0_hcp_TE_wt = rho0_TE_wt[220:250];rho0_sh_TE_wt = rho0_TE_wt[250:280]

rho0_Btin_DEN_wt = rho0_DEN_wt[0:30];rho0_fcc_DEN_wt = rho0_DEN_wt[30:60];rho0_bcc_DEN_wt = rho0_DEN_wt[60:90];rho0_cd_DEN_wt = rho0_DEN_wt[90:120];rho0_dhcp_DEN_wt = rho0_DEN_wt[120:150];rho0_Btin_hd_DEN_wt = rho0_DEN_wt[150:180];rho0_rand4_DEN_wt = rho0_DEN_wt[180:185];rho0_rand8_DEN_wt = rho0_DEN_wt[185:190];rho0_bct5_DEN_wt = rho0_DEN_wt[190:220];rho0_hcp_DEN_wt = rho0_DEN_wt[220:250];rho0_sh_DEN_wt = rho0_DEN_wt[250:280]

In [29]:
rho0_fcc_TE_wt= [0.09673084,0.09021482,0.08449455,0.07952551,0.0751474 ,0.07119835,0.06768235,0.06452084,0.06163877,0.0590039 ,0.05640183,0.05364946,0.05005879,0.04647901,0.04368379,0.04189186,0.04072559,0.03974306,0.03875802,0.0377161 ,0.03674104,0.03577425,0.03483485,0.03391843,0.03304381,0.0321998 ,0.03140109,0.03063086,0.02989303,0.02919675]

# rho0_bct5_DEN_wt = [0.06900083,0.06255148,0.05745174,0.05293614,0.04895279,0.04262975,0.0389589 ,0.03619192,0.03375811,0.03177499,0.03022887,0.02891853,0.02840704,0.02788642,0.02790464,0.02908319,0.03084631,0.03255065,0.03191184,0.03371278,0.03336661,0.03390128,0.03446173,0.03473932,0.03516789,0.03531672,0.03486211,0.03582237,0.03545685,0.03604892]

rho0_bct5_TE_wt = [0.06533168, 0.05630981, 0.04473504, 0.03978841, 0.03651514,0.03423538, 0.03246512, 0.03103024, 0.02983059, 0.02882025,0.02796016, 0.02722429, 0.02656433, 0.02595726, 0.02544661,0.02502034, 0.02466189, 0.02436471, 0.02411465, 0.02389834,0.02370005, 0.02352027, 0.02334777, 0.02317923, 0.02301507,0.02285681, 0.02270007, 0.02254006, 0.02238082, 0.02221971]

rho0_bct5_KE_wt = np.load("/Users/valeria/Documents/aiWT/Final_version/wt/wt/ks_calculations/Si_bct5/DATA2/rho0_dT.npy")

In [30]:
rho0_TE = [rho0_Btin_TE_wt, rho0_fcc_TE_wt, rho0_bcc_TE_wt, rho0_cd_TE_wt, rho0_dhcp_TE_wt, rho0_hcp_TE_wt, rho0_bct5_TE_wt, rho0_sh_TE_wt]
rho0_KE = [rho0_Btin_KE_wt, rho0_fcc_KE_wt, rho0_bcc_KE_wt, rho0_cd_KE_wt, rho0_dhcp_KE_wt, rho0_hcp_KE_wt, rho0_bct5_KE_wt, rho0_sh_KE_wt]
rho0_DEN = [rho0_Btin_DEN_wt, rho0_fcc_DEN_wt, rho0_bcc_DEN_wt, rho0_cd_DEN_wt, rho0_dhcp_DEN_wt, rho0_hcp_DEN_wt, rho0_bct5_DEN_wt, rho0_sh_DEN_wt]

In [31]:
path_pp='/Users/valeria/Documents/PP/ofpp/EAC/upf/blps/'
file='si.lda.upf'
PP_list = {'Si': path_pp+file}

In [32]:
# dictionary = {
#     'Si_Btin': {'file': 'Si_Btin.vasp'}, 
#     'Si_fcc': {'file': 'Si_fcc.vasp'},
#     'Si_bcc': {'file': 'Si_bcc.vasp'},
#     'Si_cd': {'file': 'Si_cd.vasp'},
#     'Si_dhcp': {'file': 'Si_dhcp.vasp'},
#     'Si_hcp': {'file': 'Si_hcp.vasp'},
#     'Si_bct5': {'file': 'Si_bct5.vasp'},
#     'Si_sh': {'file': 'Si_sh.vasp'},
#     'Si_cbcc': {'file': 'Si_cbcc.vasp'},
# }
dictionary = {
    'Si_Btin': {'file': 'Si_Btin.vasp', 'structure': '', 'lattice':None,  'index': 9}, 
    'Si_fcc': {'file': 'Si_fcc.vasp', 'structure': 'fcc', 'lattice': 3.405,  'index': 16},
    'Si_bcc': {'file': 'Si_bcc.vasp', 'structure': 'bcc', 'lattice': None,  'index': 9},
    'Si_cd': {'file': 'Si_cd.vasp', 'structure': 'diamond', 'lattice':5.43,  'index': 10},
    'Si_dhcp': {'file': 'Si_dhcp.vasp', 'structure': 'dhcp', 'lattice':None,  'index': 10},
    'Si_hcp': {'file': 'Si_hcp.vasp', 'structure': 'hcp', 'lattice':None,  'index': 10},
    'Si_bct5': {'file': 'Si_bct5.vasp', 'structure': 'bct5', 'lattice':None,  'index': 10},
    'Si_sh': {'file': 'Si_sh.vasp', 'structure': 'sh', 'lattice':None,  'index': 9},
}

In [57]:
def get_rho_ini(phase):
    inputfile = path_file+'/Results/Structures/'+dictionary[phase]['file']
    ions = Ions.from_ase(read(inputfile, format='vasp'))
    nr = ecut2nr(ecut=25, lattice=ions.cell)
    grid = DirectGrid(lattice=ions.cell, nr=[32, 32, 60])

    pseudo = Functional(type='PSEUDO', grid = grid, ions=ions, PP_list=PP_list)
    rho_ini = DirectField(grid=grid)
    rho_ini[:] = ions.get_ncharges()/ions.cell.volume
    return rho_ini, grid, pseudo, ions


# def get_rho_ini(phase, r):
#     if phase=='Si_fcc' or phase=='Si_8cd':
#         print(dictionary[phase]['structure'])
#         ions = Ions.from_ase(bulk('Si', str(dictionary[phase]['structure']), a= dictionary[phase]['lattice'], cubic=True))
# #         print(l[i])
#         cell = ions.get_cell()
#         ions.set_cell(cell * r, scale_atoms=True) 
#     if phase=='Si_dhcp':
#         inputfile = path_file+'/Results/Structures/'+dictionary[phase]['file']
#         ions = Ions.from_ase(read(inputfile, format='vasp'))
#         cell = ions.get_cell()
#         ions.set_cell(cell * r, scale_atoms=True) 
#     else:
#         inputfile = path_file+'/Results/Structures/'+dictionary[phase]['file']
#         ions = Ions.from_ase(read(inputfile, format='vasp'))
#         cell = ions.get_cell()
#         ions.set_cell(cell * r, scale_atoms=True)
        
#     nr = ecut2nr(ecut=25, lattice=ions.cell)
#     grid = DirectGrid(lattice=ions.cell, nr=[32, 32, 60])

#     pseudo = Functional(type='PSEUDO', grid = grid, ions=ions, PP_list=PP_list)
#     rho_ini = DirectField(grid=grid)
#     rho_ini[:] = ions.get_ncharges()/ions.cell.volume
#     return rho_ini, grid, pseudo, ions
#     return ions

In [58]:
r = np.linspace(0.8, 1.4, 30)

In [61]:
Phases = ['Si_Btin', 'Si_fcc', 'Si_bcc', 'Si_cd', 'Si_dhcp', 'Si_hcp', 'Si_bct5', 'Si_sh']
# Phases = ['Si_sh']
Rho_ini = []
Grid = []
Pseudo = []
IONS = []
for i,phase in enumerate(Phases):
    rho_ini, grid, pseudo, ions = get_rho_ini(phase)#, r[dictionary[phase]['index']])
    Rho_ini.append(rho_ini)
    Grid.append(grid)
    Pseudo.append(pseudo)
    IONS.append(ions)

setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf


### Define level of theory (XC, KEDF, etc)

In [62]:
class ExternalPotential(AbstractFunctional):
    
    def __init__(self, potential, name='EXT', type='EXT', **kwargs):
        self.potential = potential
        self.name = name
        self.type = type
        
    def compute(self, density, **kwargs):
        energy = np.sum(density*self.potential)*density.grid.dV
        functional=FunctionalOutput(name=self.name, potential=self.potential, energy=energy)
        return functional

In [63]:
A = 0.01
q = np.zeros(3)
j_h=5
V = []
Vext = []
Q = []
for i,phase in enumerate(Phases):
#     if phase=='Si_hcp' or phase=='Si_sh':
#         q = np.zeros(3)
#         q[0] = 0
#         q[1] = 2*np.pi / Grid[i].cell.lengths()[1]
#         q[2] = 0
#     elif phase=='Si_Btin' or phase=='Si_fcc' or phase=='Si_bcc' or phase=='Si_cd' or phase=='Si_dhcp' or phase=='Si_bct5':
#         q = np.zeros(3)
    q[0] = 2*np.pi / Grid[i].cell.lengths()[0]
#     q[1] = 0
#     q[2] = 0
    Q.append(q)
    v = 2 * A * np.cos(np.einsum('i,ijkl->jkl', j_h*q, Grid[i].r))
    
    vext = ExternalPotential(v)
    V.append(v)
    Vext.append(vext)

### Optimize the density

In [64]:
xc = Functional(type='XC',name='LDA')
hartree = Functional(type='HARTREE')



In [65]:
def get_response(ions, rho_ini, pseudo, rho0, vext):
        ke = Functional(type='KEDF',name='WT', rho0= rho0)
        evaluator = TotalFunctional(KE=ke, XC=xc, HARTREE=hartree, PSEUDO=pseudo)
        optimization_options = {'econv' : 1e-7}
        opt = Optimization(EnergyEvaluator=evaluator, optimization_options = optimization_options,
                optimization_method = 'TN')

        Rho_opt = opt.optimize_rho(guess_rho=rho_ini)
#         rho_opt.append(Rho_opt)
        energy_opt = evaluator.Energy(rho=Rho_opt, ions=ions)
        ke_opt = ke(Rho_opt).energy
        ### Total functional with vext

        
        totalfunctional_vext = TotalFunctional(KE=ke, XC=xc, HARTREE=hartree, PSEUDO=pseudo, EXT = vext)
        totalfunctional = TotalFunctional(KE=ke, XC=xc, HARTREE=hartree, PSEUDO=pseudo)
        optimization_options = {
            'econv' : 1e-7, # Energy Convergence (a.u./atom)
            'maxfun' : 50,   # For TN method, it's the max steps for searching direction
            'maxiter' : 50, # The max steps for optimization
            }

        opt_vext = Optimization(EnergyEvaluator=totalfunctional_vext, optimization_options = optimization_options,
            optimization_method = 'TN')
        Rho_vext = opt_vext.optimize_rho(guess_rho=Rho_opt)
        energy_vext = totalfunctional.Energy(rho=Rho_vext, ions=ions)
        ke_vext = ke(Rho_vext).energy
#         rho_vext.append(Rho_vext)
        toten_diff = -energy_opt+energy_vext
        kin_diff = -ke_opt+ke_vext
        rho_diff = 0.5 * np.abs((Rho_opt-Rho_vext)).integral()
        return kin_diff, toten_diff, rho_diff

In [66]:
diff_KIN = {}
diff_ENE = {}
diff_DEN = {}
for i,phase in enumerate(Phases):

    diff_KIN[phase] = get_response(IONS[i], Rho_ini[i], Pseudo[i],  rho0_KE[i][10], Vext[i])
    diff_ENE[phase] = get_response(IONS[i], Rho_ini[i], Pseudo[i],  rho0_TE[i][10], Vext[i])
    diff_DEN[phase] = get_response(IONS[i], Rho_ini[i], Pseudo[i], rho0_DEN[i][10], Vext[i])

Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -1.516417275998E+01     -1.516417E+01   2.736092E+00    1       1       2.592421E-02    
1       -1.585600144725E+01     -6.918287E-01   8.175036E-01    15      1       1.886401E-01    
2       -1.591054861868E+01     -5.454717E-02   7.163729E-01    14      1       3.288772E-01    
3       -1.607851220282E+01     -1.679636E-01   2.941865E-01    17      2       4.940140E-01    
4       -1.608809346916E+01     -9.581266E-03   4.811384E-02    14      1       6.266303E-01    
5       -1.609041102093E+01     -2.317552E-03   2.316556E-02    12      1       7.395201E-01    
6       -1.609124187797E+01     -8.308570E-04   5.083303E-03    14      1       8.700991E-01    
7       -1.609155774741E+01     -3.158694E-04   3.514952E-03    13      1       9.895360E-01    
8       -1.609169090914E+01     -1.331617E-04   1.668104E-03    15      1       1.121975E+00    
9       -1.609180862319E+01   

3       -1.595232582794E+01     -5.531256E-03   3.618336E-02    15      1       3.040898E-01    
4       -1.595426821116E+01     -1.942383E-03   2.473647E-03    18      1       4.680319E-01    
5       -1.595430631326E+01     -3.810210E-05   1.994060E-04    7       1       5.435789E-01    
6       -1.595431266557E+01     -6.352306E-06   1.930143E-05    14      1       6.902940E-01    
7       -1.595431367993E+01     -1.014364E-06   6.940627E-06    13      1       8.071198E-01    
8       -1.595431389711E+01     -2.171794E-07   9.004748E-07    13      1       9.196298E-01    
9       -1.595431394718E+01     -5.007099E-08   3.946903E-07    16      1       1.056465E+00    
10      -1.595431396150E+01     -1.431825E-08   7.886150E-08    14      1       1.190519E+00    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.6155258146914172
Chemical potential (eV)  : 16.74931055153129
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)     

1       -7.978371963749E+00     -2.811696E-01   3.051878E-01    17      1       1.597929E-01    
2       -7.998519546530E+00     -2.014758E-02   2.656076E-01    12      1       2.633967E-01    
3       -8.006858617101E+00     -8.339071E-03   1.697347E-01    12      1       3.728578E-01    
4       -8.030919865488E+00     -2.406125E-02   6.731818E-02    15      2       5.088336E-01    
5       -8.033165233726E+00     -2.245368E-03   4.557047E-03    13      1       6.221368E-01    
6       -8.033462184138E+00     -2.969504E-04   2.423773E-03    15      1       7.561626E-01    
7       -8.033540118933E+00     -7.793479E-05   2.995387E-04    17      1       9.006159E-01    
8       -8.033572040519E+00     -3.192159E-05   2.652911E-05    23      2       1.105646E+00    
9       -8.033572204294E+00     -1.637746E-07   2.387358E-06    5       1       1.155985E+00    
10      -8.033572245864E+00     -4.157012E-08   1.914935E-07    9       1       1.240238E+00    
11      -8.033572252133E+00   

6       -3.224262901239E+01     -1.875193E-09   6.070365E-09    8       1       5.761642E-01    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.19859195262417473
Chemical potential (eV)  : 5.40396228418927
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -2.771674798916E+01     -2.771675E+01   7.155547E+00    1       1       8.862972E-03    
1       -3.213820705444E+01     -4.421459E+00   4.257835E+00    14      2       1.380351E-01    
2       -3.222841196267E+01     -9.020491E-02   3.487009E-01    5       1       1.878471E-01    
3       -3.224410515771E+01     -1.569320E-02   2.182322E-02    8       1       2.664282E-01    
4       -3.224547845124E+01     -1.373294E-03   1.830463E-03    10      1       3.548663E-01    
5       -3.224559647867E+01     -1.180274E-04   1.480603E-04    12      1       4.613943E-01    
6       -3.224560272230E+01     -6.243638E-06   1.316913E-05    10      1       5.498183E-01

11      -1.605972142711E+01     -8.499454E-06   2.575846E-05    15      1       1.434724E+00    
12      -1.605972259874E+01     -1.171638E-06   2.351671E-06    16      1       1.582826E+00    
13      -1.605972264185E+01     -4.310561E-08   1.022812E-06    12      1       1.693063E+00    
14      -1.605972274271E+01     -1.008620E-07   4.400091E-07    12      2       1.813592E+00    
15      -1.605972277425E+01     -3.153497E-08   1.061769E-07    16      1       1.968864E+00    
16      -1.605972277899E+01     -4.738055E-09   1.611199E-08    14      1       2.092113E+00    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.20866742120250467
Chemical potential (eV)  : 5.678129749050589
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -1.605972384528E+01     -1.605972E+01   3.199973E-03    1       1       8.365154E-03    
1       -1.606006405005E+01     -3.402048E-04   1.948838E-04    4       1       5.143404E-0

11      -8.032248219024E+00     -1.604371E-05   4.498945E-05    18      5       1.570822E+00    
12      -8.032249448075E+00     -1.229052E-06   3.862950E-06    13      1       1.698444E+00    
13      -8.032249577388E+00     -1.293124E-07   3.273881E-07    17      1       1.856578E+00    
14      -8.032249588786E+00     -1.139830E-08   1.255271E-07    12      1       1.975893E+00    
15      -8.032249595676E+00     -6.889733E-09   1.175205E-08    25      2       2.232500E+00    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.407646455011033
Chemical potential (eV)  : 11.092625048770074
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -8.033678918011E+00     -8.033679E+00   1.599674E-03    1       1       1.091814E-02    
1       -8.033780815883E+00     -1.018979E-04   1.779607E-04    14      1       1.515772E-01    
2       -8.033784898712E+00     -4.082829E-06   3.969299E-05    15      1       2.986691E-01

4       -3.985053373002E+00     -2.980288E-04   2.076149E-01    12      1       5.192728E-01    
5       -3.985305850602E+00     -2.524776E-04   1.754899E-01    14      1       6.509969E-01    
6       -3.985533352314E+00     -2.275017E-04   1.801487E-01    18      1       8.077738E-01    
7       -3.985745753314E+00     -2.124010E-04   1.578943E-01    16      1       9.504280E-01    
8       -4.022458760030E+00     -3.671301E-02   4.622741E-02    28      5       1.224296E+00    
9       -4.022812498600E+00     -3.537386E-04   5.561332E-03    13      1       1.345577E+00    
10      -4.022956650614E+00     -1.441520E-04   2.037956E-03    16      2       1.494753E+00    
11      -4.022970845923E+00     -1.419531E-05   4.016803E-04    14      1       1.619680E+00    
12      -4.022994397682E+00     -2.355176E-05   1.110396E-04    20      2       1.807468E+00    
13      -4.022999140293E+00     -4.742612E-06   5.535258E-05    15      1       1.940422E+00    
14      -4.023000242754E+00   

2       -4.022638518181E+00     -5.219460E-06   7.731101E-05    16      1       2.776861E-01    
3       -4.022639634111E+00     -1.115930E-06   2.665768E-05    12      1       3.871489E-01    
4       -4.022639703741E+00     -6.963001E-08   1.423761E-05    15      1       5.234060E-01    
5       -4.022639721086E+00     -1.734439E-08   7.360959E-06    14      1       6.606610E-01    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.36571648908322646
Chemical potential (eV)  : 9.951652559919964


# N/V

In [67]:
rho_opt_nv = []
rho_vext_nv = []
toten_diff_nv = []
kin_diff_nv = []
rho0_nv = []
for i,phase in enumerate(Phases):
        ke = Functional(type='KEDF',name='WT', rho0= Rho_ini[i].mean())
        rho0_nv.append(Rho_ini[i].mean())
        print('rho0'+str(phase)+': ', Rho_ini[i].mean())
        evaluator = TotalFunctional(KE=ke, XC=xc, HARTREE=hartree, PSEUDO=Pseudo[i])
        optimization_options = {'econv' : 1e-7}
        opt = Optimization(EnergyEvaluator=evaluator, optimization_options = optimization_options,
                optimization_method = 'TN')

        Rho_opt = opt.optimize_rho(guess_rho=Rho_ini[i])
        rho_opt_nv.append(Rho_opt)
        energy_opt = evaluator.Energy(rho=Rho_opt, ions=IONS[i])
        ke_opt = ke(Rho_opt).energy
        
        totalfunctional_vext = TotalFunctional(KE=ke, XC=xc, HARTREE=hartree, PSEUDO=Pseudo[i], EXT = Vext[i])
        totalfunctional = TotalFunctional(KE=ke, XC=xc, HARTREE=hartree, PSEUDO=Pseudo[i])

        optimization_options = {
            'econv' : 1e-7, # Energy Convergence (a.u./atom)
            'maxfun' : 50,   # For TN method, it's the max steps for searching direction
            'maxiter' : 50, # The max steps for optimization
            }

        opt_vext = Optimization(EnergyEvaluator=totalfunctional_vext, optimization_options = optimization_options,
            optimization_method = 'TN')
        Rho_vext = opt_vext.optimize_rho(guess_rho=Rho_opt)
        energy_vext = totalfunctional.Energy(rho=Rho_vext, ions=IONS[i])
        ke_vext = ke(Rho_vext).energy
        rho_vext_nv.append(Rho_vext)
        toten_diff_nv.append(-energy_opt+energy_vext)
        kin_diff_nv.append(-ke_opt+ke_vext)

rho_diff_nv = []
rho_diff_abs_nv = []
for i in np.arange(0,len(Phases)):
    Rho_diff = rho_vext_nv[i] - rho_opt_nv[i]
#     rho_diff_nv.append(Rho_diff)
    rho_abs = np.abs(Rho_diff).integral()*0.5
    rho_diff_nv.append(rho_abs)

rho0Si_Btin:  0.03867117375086124
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -1.516417275998E+01     -1.516417E+01   2.736092E+00    1       1       1.141787E-02    
1       -1.585349173287E+01     -6.893190E-01   7.994566E-01    15      1       1.778610E-01    
2       -1.590647596374E+01     -5.298423E-02   6.912261E-01    14      1       3.418920E-01    
3       -1.606330741614E+01     -1.568315E-01   2.910144E-01    17      2       5.213718E-01    
4       -1.607154991237E+01     -8.242496E-03   3.379199E-02    14      1       6.529939E-01    
5       -1.607324619442E+01     -1.696282E-03   1.733908E-02    12      1       7.706997E-01    
6       -1.607382998395E+01     -5.837895E-04   3.236409E-03    16      1       9.162629E-01    
7       -1.607383801220E+01     -8.028250E-06   2.253069E-03    13      1       1.046000E+00    
8       -1.607384342259E+01     -5.410386E-06   1.872160E-03    12      1       1.162154E+00 

9       -3.200094044309E+01     -3.150305E-09   8.075224E-09    8       1       9.660201E-01    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.2045174914137919
Chemical potential (eV)  : 5.565204407595905
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -3.200094044309E+01     -3.200094E+01   6.400008E-03    1       1       1.118636E-02    
1       -3.200250007642E+01     -1.559633E-03   2.107135E-04    1       1       4.078126E-02    
2       -3.200255456380E+01     -5.448738E-05   3.272131E-05    12      1       1.673701E-01    
3       -3.200255691439E+01     -2.350593E-06   5.059427E-06    12      1       2.782631E-01    
4       -3.200255725089E+01     -3.364986E-07   5.024607E-07    13      1       3.981011E-01    
5       -3.200255728662E+01     -3.573088E-08   8.523123E-08    12      1       5.150840E-01    
6       -3.200255729512E+01     -8.495050E-09   8.695100E-09    16      1       6.590290E-01

5       -1.602595342820E+01     -4.222603E-09   2.450794E-08    13      1       4.997211E-01    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.2868451650207955
Chemical potential (eV)  : 7.8054545146041905
rho0Si_sh:  0.04041054967600243
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -3.801072869396E+00     -3.801073E+00   7.011073E-01    1       1       1.069331E-02    
1       -3.970545457360E+00     -1.694726E-01   1.992162E-01    14      1       1.392422E-01    
2       -3.983148999736E+00     -1.260354E-02   3.401914E-01    14      1       2.656732E-01    
3       -3.983592259542E+00     -4.432598E-04   2.074898E-01    12      1       3.822312E-01    
4       -3.983878249600E+00     -2.859901E-04   1.992452E-01    12      1       4.986351E-01    
5       -3.984120221213E+00     -2.419716E-04   1.678938E-01    14      1       6.300821E-01    
6       -3.984338062454E+00     -2.178412E-04   1.725980E-0

In [68]:
ke_diff_ke  = {}
ke_diff_te  = {}
ke_diff_den = {}
te_diff_ke  ={} 
te_diff_te  ={} 
te_diff_den ={}
rho_diff_ke  ={} 
rho_diff_te  ={} 
rho_diff_den ={}

for i, p in enumerate(Phases):
    ke_diff_ke[p]  = diff_KIN[p][0]/A
    ke_diff_te[p]  = diff_ENE[p][0]/A
    ke_diff_den[p] = diff_DEN[p][0]/A
    
    te_diff_ke[p]  = diff_KIN[p][1]/A
    te_diff_te[p]  = diff_ENE[p][1]/A
    te_diff_den[p] = diff_DEN[p][1]/A
    
    rho_diff_ke[p]  = diff_KIN[p][2]/A
    rho_diff_te[p]  = diff_ENE[p][2]/A
    rho_diff_den[p] = diff_DEN[p][2]/A

# Response table

In [69]:
ks_ke_diff_k8 = np.load('/Users/valeria/Documents/aiWT/Final_version/wt/wt/Results/Response/ks_ke_diff_j5.npy')
ks_te_diff_k8 = np.load('/Users/valeria/Documents/aiWT/Final_version/wt/wt/Results/Response/ks_te_diff_j5.npy')
ks_rho_diff_k8 = np.load('/Users/valeria/Documents/aiWT/Final_version/wt/wt/Results/Response/ks_rho_diff_j5.npy')

In [70]:
from tabulate import tabulate
## Energies
data = [
    ["Btin", ke_diff_ke['Si_Btin']/4, ke_diff_te['Si_Btin']/4, ke_diff_den['Si_Btin']/4,   kin_diff_nv[0]/(A*4), ks_ke_diff_k8[0]/(A*4*2)],
    ["bcc",  ke_diff_ke['Si_bcc']/2,  ke_diff_te['Si_bcc']/2,  ke_diff_den['Si_bcc']/2,    kin_diff_nv[2]/(A*2), ks_ke_diff_k8[2]/(A*2*2)],
    ["fcc",  ke_diff_ke['Si_fcc']/4,  ke_diff_te['Si_fcc']/4,   ke_diff_den['Si_fcc']/4,   kin_diff_nv[1]/(A*4), ks_ke_diff_k8[1]/(A*4*2)],
    ["hcp",  ke_diff_ke['Si_hcp']/2,  ke_diff_te['Si_hcp']/2,  ke_diff_den['Si_hcp']/2,    kin_diff_nv[5]/(A*2), ks_ke_diff_k8[5]/(A*2*2)],
    ["bct5", ke_diff_ke['Si_bct5']/4, ke_diff_te['Si_bct5']/4, ke_diff_den['Si_bct5']/4,   kin_diff_nv[6]/(A*4), ks_ke_diff_k8[6]/(A*4*2)],
    ["sh",   ke_diff_ke['Si_sh'],   ke_diff_te['Si_sh'], ke_diff_den['Si_sh'],             kin_diff_nv[7]/(A*1), ks_ke_diff_k8[7]/(A*2)],
    ["cd",   ke_diff_ke['Si_cd']/8,   ke_diff_te['Si_cd']/8,  ke_diff_den['Si_cd']/8,      kin_diff_nv[3]/(A*8), ks_ke_diff_k8[3]/(A*8*2)],
    ["dhcp", ke_diff_ke['Si_dhcp']/4,   ke_diff_te['Si_dhcp']/4,  ke_diff_den['Si_dhcp']/4,kin_diff_nv[4]/(A*4), ks_ke_diff_k8[4]/(A*4*2)],
]

table = tabulate(data, headers=[r"KE diff","Response \n rho0_KIN","Response \n rho0_TE","Response \n rho0_DEN", "Response \n N/V", "KS k=8"], tablefmt="grid")

print(table)
# print(tabulate(data, tablefmt="latex"))
print(tabulate(data, tablefmt="latex"))

+-----------+-------------+-------------+-------------+-------------+-------------+
| KE diff   |   Response  |   Response  |   Response  |   Response  |      KS k=8 |
|           |    rho0_KIN |     rho0_TE |    rho0_DEN |         N/V |             |
+===========+=============+=============+=============+=============+=============+
| Btin      |  0.0163704  |  0.0219885  |  0.017159   |  0.0165425  |  0.0157613  |
+-----------+-------------+-------------+-------------+-------------+-------------+
| bcc       |  0.00659331 |  0.00668215 |  0.00744673 |  0.00696171 |  0.00578817 |
+-----------+-------------+-------------+-------------+-------------+-------------+
| fcc       |  0.00817308 |  0.00816876 |  0.00815049 |  0.00727228 |  0.00727207 |
+-----------+-------------+-------------+-------------+-------------+-------------+
| hcp       | -0.0240711  | -0.022414   | -0.0157653  | -0.0186104  | -0.0250452  |
+-----------+-------------+-------------+-------------+-------------+-------

In [71]:
from tabulate import tabulate
## Energies
data = [
    ["Btin", te_diff_ke['Si_Btin']/4, te_diff_te['Si_Btin']/4, te_diff_den['Si_Btin']/4, toten_diff_nv[0]/(A*4), ks_te_diff_k8[0]/(A*4)],
    ["bcc",  te_diff_ke['Si_bcc']/2,  te_diff_te['Si_bcc']/2,  te_diff_den['Si_bcc']/2,  toten_diff_nv[2]/(A*2), ks_te_diff_k8[2]/(A*2)],
    ["fcc",  te_diff_ke['Si_fcc']/4,  te_diff_te['Si_fcc']/4,  te_diff_den['Si_fcc']/4,  toten_diff_nv[1]/(A*4), ks_te_diff_k8[1]/(A*4)],
    ["hcp",  te_diff_ke['Si_hcp']/2,  te_diff_te['Si_hcp']/2,  te_diff_den['Si_hcp']/2,  toten_diff_nv[5]/(A*2), ks_te_diff_k8[5]/(A*2)],
    ["bct5", te_diff_ke['Si_bct5']/4, te_diff_te['Si_bct5']/4, te_diff_den['Si_bct5']/4, toten_diff_nv[6]/(A*4), ks_te_diff_k8[6]/(A*4)],
    ["sh",   te_diff_ke['Si_sh'],     te_diff_te['Si_sh'],     te_diff_den['Si_sh'],     toten_diff_nv[7]/(A*1), ks_te_diff_k8[7]/A],
    ["cd",   te_diff_ke['Si_cd']/8,   te_diff_te['Si_cd']/8,   te_diff_den['Si_cd']/8,   toten_diff_nv[3]/(A*8), ks_te_diff_k8[3]/(A*8)],
    ["dhcp", te_diff_ke['Si_dhcp']/4, te_diff_te['Si_dhcp']/4, te_diff_den['Si_dhcp']/4, toten_diff_nv[4]/(A*4), ks_te_diff_k8[4]/(A*4)],
    
]

table = tabulate(data, headers=[r"TE diff","Response \n rho0_KIN","Response \n rho0_TE","Response \n rho0_DEN", "Response \n N/V", "KS k=8"], tablefmt="grid")

print(table)
# print(tabulate(data, tablefmt="latex"))
print(tabulate(data, tablefmt="latex"))

+-----------+-------------+-------------+-------------+-------------+------------+
| TE diff   |   Response  |   Response  |   Response  |   Response  |     KS k=8 |
|           |    rho0_KIN |     rho0_TE |    rho0_DEN |         N/V |            |
+===========+=============+=============+=============+=============+============+
| Btin      |  0.0153931  |  0.0153781  |  0.0153819  |  0.0154056  | 0.0152253  |
+-----------+-------------+-------------+-------------+-------------+------------+
| bcc       |  0.00584154 |  0.00583946 |  0.00584121 |  0.00583882 | 0.00582775 |
+-----------+-------------+-------------+-------------+-------------+------------+
| fcc       |  0.00726917 |  0.00726923 |  0.00726947 |  0.00727054 | 0.00726057 |
+-----------+-------------+-------------+-------------+-------------+------------+
| hcp       |  0.00533757 |  0.00538004 |  0.00532553 |  0.00520384 | 0.00493791 |
+-----------+-------------+-------------+-------------+-------------+------------+
| bc

In [72]:
ks_rho_sh = 0.03267848
# print(ke_diff) [0.01339731]
# print(te_diff) [0.00025155]
# print(rho_diff)[0.03267848]

In [73]:
ks_rho_diff_k8[7]

0.004479951777319948

In [74]:
from tabulate import tabulate
## Energies
data = [
    ["Btin", rho_diff_ke['Si_Btin']/4, rho_diff_te['Si_Btin']/4, rho_diff_den['Si_Btin']/4, rho_diff_nv[0]/(A*4), ks_rho_diff_k8[0]/(A*4)],
    ["bcc",  rho_diff_ke['Si_bcc']/2,  rho_diff_te['Si_bcc']/2,  rho_diff_den['Si_bcc']/2,  rho_diff_nv[2]/(A*2), ks_rho_diff_k8[2]/(A*2)],
    ["fcc",  rho_diff_ke['Si_fcc']/4,  rho_diff_te['Si_fcc']/4,  rho_diff_den['Si_fcc']/4,  rho_diff_nv[1]/(A*4), ks_rho_diff_k8[1]/(A*4)],
    ["hcp",  rho_diff_ke['Si_hcp']/2,  rho_diff_te['Si_hcp']/2,  rho_diff_den['Si_hcp']/2,  rho_diff_nv[5]/(A*2), ks_rho_diff_k8[5]/(A*2)],
    ["bct5", rho_diff_ke['Si_bct5']/4, rho_diff_te['Si_bct5']/4, rho_diff_den['Si_bct5']/4, rho_diff_nv[6]/(A*4), ks_rho_diff_k8[6]/(A*4)],
    ["sh",   rho_diff_ke['Si_sh'],     rho_diff_te['Si_sh'],     rho_diff_den['Si_sh'],     rho_diff_nv[7]/(A*1), ks_rho_diff_k8[7]/(A)],
    ["cd",   rho_diff_ke['Si_cd']/8,   rho_diff_te['Si_cd']/8,   rho_diff_den['Si_cd']/8,   rho_diff_nv[3]/(A*8), ks_rho_diff_k8[3]/(A*8)],
    ["dhcp", rho_diff_ke['Si_dhcp']/4, rho_diff_te['Si_dhcp']/4, rho_diff_den['Si_dhcp']/4, rho_diff_nv[4]/(A*4), ks_rho_diff_k8[4]/(A*4)],
    
]

table = tabulate(data, headers=[r"Rho diff","Response \n rho0_KIN","Response \n rho0_TE","Response \n rho0_DEN", "Response \n N/V", "KS k=8"], tablefmt="grid")

print(table)
# print(tabulate(data, tablefmt="latex"))
print(tabulate(data, tablefmt="latex"))

+------------+-------------+-------------+-------------+-------------+----------+
| Rho diff   |   Response  |   Response  |   Response  |   Response  |   KS k=8 |
|            |    rho0_KIN |     rho0_TE |    rho0_DEN |         N/V |          |
+============+=============+=============+=============+=============+==========+
| Btin       |    0.980221 |    0.980544 |    0.980359 |    0.983183 | 0.982694 |
+------------+-------------+-------------+-------------+-------------+----------+
| bcc        |    0.378054 |    0.378039 |    0.378146 |    0.377927 | 0.388398 |
+------------+-------------+-------------+-------------+-------------+----------+
| fcc        |    0.470559 |    0.470565 |    0.4706   |    0.470595 | 0.46871  |
+------------+-------------+-------------+-------------+-------------+----------+
| hcp        |    0.414555 |    0.416177 |    0.406958 |    0.40542  | 0.402098 |
+------------+-------------+-------------+-------------+-------------+----------+
| bct5       |  